# Spark Learning Note - MLlib
Jia Geng | gjia0214@gmail.com


In [1]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName('MLexample').getOrCreate()
spark

## 1. Classification Models in MLlib

Spark supports following classificatoin algorithms
- Logistic Regression
- Decision Trees
- Random Forest
- Gradient-boosted Trees

**Spark does not support Multi-label predictions natively**. Inorder to train multilabel model, one must train one model per label. (e.g. predict book's genre and book's author, you need to build two models seperately). 
- *Personal opinion: this makes assumption that different labels is independent each other. One way out is to combine these labels togther, e.g., genre, author -> genreauthor.*

